### CREATE A FUNCTION TO EVALUATE THE MODEL BEFORE SENDING IT ONLINE


In [2]:
# from torch.utils.data import DataLoader
# import sys
# import os 
# # Add the project root directory to Python path
# sys.path.append(os.path.dirname(os.getcwd()))
# from data.dataset import Dataset
# from models.dinov2_advanced import DinoV2Finetune_advanced
# import hydra







In [3]:
# #create the dataset with the test set
# model=DinoV2Finetune_advanced()
# checkpoint_path=[
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_04-11-10_with_Gelu.pt",
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_03-20-35_with_Gelu.pt", 
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_02-09-07_with_Gelu.pt", 
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-03_06-12-30_last.pt"

# ]
# def run_evaluation(model , checkpoint_path):
#     @hydra.main(config_path="configs", config_name="train")
#     def evaluate_model(cfg):
#         test_dataset=Dataset(cfg.datamodule.dataset_path, "test", transforms=hydra.utils.instantiate(cfg.datamodule.test_transform), metadata=cfg.datamodule.metadata)
#         test_loader=DataLoader(test_dataset, batch_size=cfg.datamodule.batch_size, shuffle=False, num_workers=cfg.datamodule.num_workers)
#         loss_fn = hydra.utils.instantiate(cfg.loss_fn)
#         model.load_state_dict(checkpoint)
#         print("Model loaded")
#         for _, batch in enumerate(test_loader):
#             batch["image"] = batch["image"].to(device)
#             batch["target"] = batch["target"].to(device).squeeze()
#             with torch.no_grad():
#                 preds = model(batch).squeeze().cpu().numpy()
#             loss = loss_fn(preds, batch["target"])
#             epoch_val_loss += loss.detach().cpu().numpy() * len(batch["image"])
#             num_samples_val += len(batch["image"])
#         epoch_val_loss /= num_samples_val
#         val_metrics["val/loss_epoch"] = epoch_val_loss
#         logger_std.info(f"----------------Epoch {epoch} val loss: {epoch_val_loss:.4f}----------------")
#     evaluate_model()



In [4]:
# for checkpoint in checkpoint_path:
#     print("--------------------------------\n")
#     print(f"Evaluating model with checkpoint: {checkpoint} \n")
#     run_evaluation(model, checkpoint)


In [5]:
# import torch
# import torch.nn as nn


# class DinoV2Finetune_large(nn.Module):
#     def __init__(self, frozen=False):
#         super().__init__()
#         self.backbone =torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg')
#         self.backbone.head = nn.Identity()
#         self.dim = self.backbone.norm.normalized_shape[0]
#         if frozen:
#             for param in self.backbone.parameters():
#                 param.requires_grad = False
#         self.regression_head = nn.Sequential(
#             nn.Linear(self.backbone.norm.normalized_shape[0], 1),
#             nn.ReLU(),
#         )

#     def forward(self, x):
#         x = self.backbone(x["image"])
#         x = self.regression_head(x)
#         return x
# class DinoV2Finetune_small(nn.Module):
#     def __init__(self, frozen=False):
#         super().__init__()
#         self.backbone =torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14_reg")
#         self.backbone.head = nn.Identity()
#         self.dim = self.backbone.norm.normalized_shape[0]
#         if frozen:
#             for param in self.backbone.parameters():
#                 param.requires_grad = False
#         self.regression_head = nn.Sequential(
#             nn.Linear(self.backbone.norm.normalized_shape[0], 1),
#             nn.ReLU(),
#         )

#     def forward(self, x):
#         x = self.backbone(x["image"])
#         x = self.regression_head(x)
#         return x

In [6]:
# large_dinov= DinoV2Finetune_large()
# small_dinov= DinoV2Finetune_small()
# #check parameters of the model
# # print(model.parameters())


In [7]:
import torch
import torch.nn as nn
from torchinfo import summary


def print_model_info(model):
    """Print detailed information about the model"""
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Frozen parameters: {total_params - trainable_params:,}")
    
    # Print model architecture
    print("\nModel Architecture:")
    print(model)
    
    # Print parameter details for each layer
    print("\nDetailed Layer Information:")
    for name, param in model.named_parameters():
        print(f"{name}: shape={param.shape}, requires_grad={param.requires_grad}")
    
    # # Print model summary with torchinfo
    # print("\nModel Summary:")
    # # Assuming input image size of 224x224 with batch size 2
    # summary(model, input_data={"image": torch.zeros(2, 3, 224, 224)})


def freeze_backbone(model, freeze=True):
    """Freeze or unfreeze the backbone parameters"""
    for param in model.backbone.parameters():
        param.requires_grad = not freeze
    return model


def replace_regression_head(model, new_layers):
    """Replace the regression head with a new sequence of layers"""
    model.regression_head = nn.Sequential(*new_layers)
    return model


def change_output_dimension(model, output_dim):
    """Change the output dimension of the regression head"""
    # Create a new regression head with the specified output dimension
    new_head = nn.Sequential(
        nn.Linear(model.dim, output_dim),
        nn.ReLU(),
    )
    model.regression_head = new_head
    return model


In [8]:
# Calculate memory usage in MB
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_in_mb = (param_size + buffer_size) / 1024**2
    return size_in_mb

# Print the model size
model_size_mb = get_model_size(model=large_dinov)
print(f"Model size large: {model_size_mb:.2f} MB")

model_size_mb = get_model_size(model=small_dinov)
print(f"Model size small: {model_size_mb:.2f} MB")


NameError: name 'large_dinov' is not defined

In [9]:
print_model_info(model=model)

NameError: name 'model' is not defined

## VISUALIZE THE DATASET

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Load the dataset

train_data=pd.read_csv("../dataset/train_val.csv")


In [11]:
train_data.head(20)

,Unnamed: 0,id,channel,title,date,description,views,year
0,0,--2s6hjGrm4,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI & VFX Breakdowns: ""Warzone"" - by Ramesh Th...",2020-12-15 05:00:01+00:00,"Check out this revealing VFX Breakdown ""Warzon...",12299,2020
1,1,--DnfroyKQ8,UC-1rx8j9Ggp8mp4uD0ZdEIA,"A Sci-Fi Short Film: ""Exit"" - by Ng King Kwan ...",2020-07-01 16:00:00+00:00,"TheCGBros Presents ""Exit"" by Ng King Kwan - Th...",7494,2020
2,2,--aiU7VQKEw,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D Animated Short: ""Lost Love"" - by Akash ...",2019-02-18 20:30:00+00:00,"TheCGBros Presents ""Lost Love"" by Akash Manack...",11831,2019
3,6,-0SrlZAvSVM,UCW6NyJ6oFLPTnx7iGRZXDDg,Jo Goes Hunting - Careful | Animated music vid...,2020-03-10 14:30:01+00:00,"On the borderless map of a magical planet, lit...",2248,2020
4,10,-13Y2Pe7kFs,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Breakdown: ""Logan (Wolverine): Digital...",2017-09-20 20:13:52+00:00,Check out this outstanding behind-the-scenes l...,113806,2017
5,12,-1ElNi3aewU,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D Animated Spot : ""Aroma"" by - Space Patrol",2013-04-09 14:28:37+00:00,Here is another great 3D animated spot created...,6228,2013
6,13,-1FZlgerlnQ,UCXLfJIEp91L-fD0gP131IaQ,Hellevate Super Short Horror Comedy | Screamfest,2021-01-29 18:00:11+00:00,Hellevate is a super short horror comedy that ...,18652,2021
7,14,-1enK4XQvow,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Live Action Short Film: ""RELATIVITY"" ...",2022-01-05 17:00:00+00:00,"TheCGBros Presents ""RELATIVITY"" by Guillaume ...",20409,2022
8,15,-1hANFUO3yc,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Short Films : ""TOB"" - by Andres Galeano",2014-03-18 00:31:30+00:00,"Watch this VFX short Film, by the talented And...",10370,2014
9,16,-1nkzRrnVfA,UCW6NyJ6oFLPTnx7iGRZXDDg,"Horror short film about a cursed cemetery | ""T...",2021-12-15 14:30:22+00:00,Sara and Ali are looking through photo's from ...,228067,2021


In [14]:
print(train_data["description"][1])
print(train_data["title"][1])

TheCGBros Presents "Exit" by Ng King Kwan - The story about a man struggling between different values in life, will he find a new perspective towards life and get out of this dilemma? For more information, please see the details and links below:

Team:
A Film by - Ng King Kwan
Music & Sound by - Li Ming Hei
3D Scanned Model - Stanford Computer Graphics Laboratory

FOLLOW US BELOW:
Ng King Kwan
https://kwanfx.com/
https://www.instagram.com/kwan_fx/
Li Ming Hei
http://andrewli.netlify.app/
andrewes.li.hk@gmail.com

We really hope you enjoy watching this video! 
When you do, please hit the LIKE ✔️ SUB ✔️ SHARE ✔️ button as well as the bell symbol 🔔 so you'll be kept in the loop as new material is published on our channel! 
▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂

Help support TheCGBros by purchasing our official Gear!!
►https://teespring.com/stores/cgb-gear

Thank You for SUBSCRIBING -►http://bit.ly/2c8Fl8u
▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂

Watch More CGI VFX Short Films HERE ►https://bit.ly/2MpDz7E

WANT TO GET 

In [20]:
import re

def clean_description(description):
    import re
    # Supprimer liens, emails, hashtags, mentions
    description = re.sub(r'https?:\/\/\S+|www\.\S+', '', description)
    description = re.sub(r'\S+@\S+', '', description)
    description = re.sub(r'#\w+', '', description)
    description = re.sub(r'@[A-Za-z0-9_]+', '', description)
    description = re.sub(r'[^\w\s,.!?\'":;()-]', '', description)
    
    # Liste augmentée des triggers d'appel à l'action (cta)
    calls_to_action = [
        "like", "subscribe", "sub", "share", "bell", "notification",
        "comment", "don't forget", "hit the", "click the", "smash the",
        "support", "checkout our merch", "thank you", "follow us", "stay tuned",
        "keep in the loop", "be kept in the loop", "published on our channel",
        "ring the bell", "please hit", "please consider", "button", "buy",
        "purchase", "get featured", "join us", "as new material is published",
        "as new content is published"
    ]
    
    lines = description.split('\n')
    cleaned_lines = []
    for line in lines:
        line_stripped = line.strip().lower()
        if any(cta in line_stripped for cta in calls_to_action):
            continue
        if line_stripped.startswith("►") or line_stripped.startswith("▪") or line_stripped.startswith("—"):
            continue
        if len(line_stripped) < 5:
            continue
        cleaned_lines.append(line.strip())
    
    description = ' '.join(cleaned_lines)
    description = re.sub(r'\s+', ' ', description).strip()
    return description

def make_prompt(entry):
    """
    entry : dict ou pd.Series avec les clés 'title', 'channel', 'date', 'description'
    => Renvoie un texte structuré et propre décrivant la vidéo
    """
    title = entry['title']
    channel = entry['channel']
    date = entry['year']  # Garde juste l'année/mois/jour si besoin
    desc_clean = clean_description(entry['description'])
    return (
        f"La vidéo \"{title}\" publiée par la chaîne {channel} en {date} parle de : {desc_clean}"
        if desc_clean else
        f"La vidéo \"{title}\" publiée par la chaîne {channel} en {date}."
    )

In [23]:
print(make_description(train_data.iloc[3]))

La vidéo "Jo Goes Hunting - Careful | Animated music video by Alice Saey" publiée par la chaîne UCW6NyJ6oFLPTnx7iGRZXDDg en 2020 parle de : On the borderless map of a magical planet, little beings pick, brush, weave, fish and collect elements from their natural environment to sustain their life as a group. This essential balance turns to chaos once they misuse their findings to polish their individual appearance. Original title - Careful Director - Alice Saey Poducers - Emmanuel-Alain Raynal, Pierre Baussaron, Dennis Platvoet Production - Autoproduction Music - Jimmi Jo Hueting Year - 2020 Jo Goes Hunting, Grand Douche Records 2019 Licensed by Miyu Distribution


In [10]:
#load the train dataset
import torch
import torchvision.transforms as transforms
from data.dataset import Dataset

train_data_path="CSC_43M04_EP_challenge/dataset"
train_transformers=transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_set=Dataset(train_data_path, "train_val", train_transformers, metadata)

ModuleNotFoundError: No module named 'data'

In [26]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Define function to load and display an image
def load_and_display_image(image_id):
    image_path = os.path.join("dataset", "train_val", f"{image_id}.jpg")
    print(f"Loading image from: {image_path}")
    
    if os.path.exists(image_path):
        img = Image.open(image_path).convert("RGB")
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image ID: {image_id}")
        plt.show()
        return img
    else:
        print(f"Error: Image file not found at {image_path}")
        return None

# Example: load the first image from your DataFrame
first_image_id = train_data['id'].iloc[0]  # Gets the ID of the first row
img = load_and_display_image(first_image_id)

Loading image from: dataset/train_val/--2s6hjGrm4.jpg
Error: Image file not found at dataset/train_val/--2s6hjGrm4.jpg


## TRY TO PERFORM INFERENCE ON THE TRAINED MODEL AND THE LARGEST MODEL

In [2]:
pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 51.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [transformers] [transformers]ub]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# ----- Config -----
pretrained_model = "prajjwal1/bert-tiny"  # Very small model!
num_classes = 2
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----- Dummy Dataset -----
texts = [
    "I love this movie!",     # pos
    "This was awesome",       # pos
    "I hate this film",       # neg
    "Terrible experience",    # neg
    "Brilliant performance",  # pos
    "Awful and boring"        # neg
]
labels = [1, 1, 0, 0, 1, 0]

class ToyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=32):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
        self.labels = torch.tensor(labels)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# ----- Custom Model -----
class MyCustomLLM(nn.Module):
    def __init__(self, base_model_path, num_classes):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_path)
        hidden_size = self.backbone.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_classes)
    def forward(self, input_ids, attention_mask):
        output = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled = output.last_hidden_state[:, 0]  # CLS token
        return self.classifier(pooled)

# ----- Prepare -----
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
dataset = ToyDataset(texts, labels, tokenizer)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

model = MyCustomLLM(pretrained_model, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ----- Training Function -----
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# ----- Inference/Prediction -----
def predict(model, texts):
    model.eval()
    encoded = tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(encoded['input_ids'], encoded['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        preds = probs.argmax(dim=1)
    return preds.cpu().tolist(), probs.cpu().tolist()



In [3]:
# ----- Before Training -----
print("Predictions before training:")
preds, probs = predict(model, texts)
for t,p in zip(texts, preds):
    print(f"{t} -> {'positive' if p==1 else 'negative'}")

# ----- Training -----
for epoch in range(5):
    loss = train_one_epoch(model, loader, optimizer, criterion)
    print(f"Epoch {epoch+1} Loss: {loss:.4f}")

# ----- After Training -----
print("\nPredictions after training:")
preds, probs = predict(model, texts)
for t,p in zip(texts, preds):
    print(f"{t} -> {'positive' if p==1 else 'negative'}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predictions before training:
I love this movie! -> negative
This was awesome -> positive
I hate this film -> negative
Terrible experience -> positive
Brilliant performance -> positive
Awful and boring -> negative
Epoch 1 Loss: 0.5483
Epoch 2 Loss: 0.6986
Epoch 3 Loss: 0.6109
Epoch 4 Loss: 0.5144
Epoch 5 Loss: 0.4636

Predictions after training:
I love this movie! -> positive
This was awesome -> positive
I hate this film -> negative
Terrible experience -> negative
Brilliant performance -> positive
Awful and boring -> negative


## TEST DE MISE A JOUR DU CSV

In [2]:
import pandas as pd

def process_youtube_csv(csv_path):
    # Chargement du fichier CSV
    df = pd.read_csv(csv_path)
    
    # ------ 1. Ajout de la colonne 'view_classes' ------
    view_thresholds = [
        0,        # 0 vues
        1000,     # 1K vues
        10000,    # 10K vues
        100000,   # 100K vues
        1000000,  # 1M vues
        float('inf')  # Infini
    ]
    labels = [
        "Hidden Gems",
        "Rising Stars",
        "Solid Performers",
        "Viral Hits",
        "Mega Blockbusters"
    ]
    def assign_view_class(views):
        for i in range(len(view_thresholds) - 1):
            if view_thresholds[i] <= views < view_thresholds[i+1]:
                return labels[i]
        return labels[-1]
    df['view_classes'] = df['views'].apply(assign_view_class)
    
    # ------ 2. Ajout de la colonne 'channel_real_name' ------
    if 'channel' in df.columns:
        unique_channels = df['channel'].unique()
        channel_mapping = {channel: f"channel{i+1}" for i, channel in enumerate(unique_channels)}
        df['channel_real_name'] = df['channel'].map(channel_mapping)
    
    # Retourner le DataFrame modifié
    return df



In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Load the dataset

# train_data=pd.read_csv("../dataset/train_val.csv")

data_processed=process_youtube_csv("../dataset/train_val.csv")

NameError: name 'process_youtube_csv' is not defined

In [4]:
data_processed.head(20)


,Unnamed: 0,id,channel,title,date,description,views,year,view_classes,channel_real_name
0,0,--2s6hjGrm4,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI & VFX Breakdowns: ""Warzone"" - by Ramesh Th...",2020-12-15 05:00:01+00:00,"Check out this revealing VFX Breakdown ""Warzon...",12299,2020,Solid Performers,channel1
1,1,--DnfroyKQ8,UC-1rx8j9Ggp8mp4uD0ZdEIA,"A Sci-Fi Short Film: ""Exit"" - by Ng King Kwan ...",2020-07-01 16:00:00+00:00,"TheCGBros Presents ""Exit"" by Ng King Kwan - Th...",7494,2020,Rising Stars,channel1
2,2,--aiU7VQKEw,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D Animated Short: ""Lost Love"" - by Akash ...",2019-02-18 20:30:00+00:00,"TheCGBros Presents ""Lost Love"" by Akash Manack...",11831,2019,Solid Performers,channel1
3,6,-0SrlZAvSVM,UCW6NyJ6oFLPTnx7iGRZXDDg,Jo Goes Hunting - Careful | Animated music vid...,2020-03-10 14:30:01+00:00,"On the borderless map of a magical planet, lit...",2248,2020,Rising Stars,channel2
4,10,-13Y2Pe7kFs,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Breakdown: ""Logan (Wolverine): Digital...",2017-09-20 20:13:52+00:00,Check out this outstanding behind-the-scenes l...,113806,2017,Viral Hits,channel1
5,12,-1ElNi3aewU,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D Animated Spot : ""Aroma"" by - Space Patrol",2013-04-09 14:28:37+00:00,Here is another great 3D animated spot created...,6228,2013,Rising Stars,channel1
6,13,-1FZlgerlnQ,UCXLfJIEp91L-fD0gP131IaQ,Hellevate Super Short Horror Comedy | Screamfest,2021-01-29 18:00:11+00:00,Hellevate is a super short horror comedy that ...,18652,2021,Solid Performers,channel3
7,14,-1enK4XQvow,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Live Action Short Film: ""RELATIVITY"" ...",2022-01-05 17:00:00+00:00,"TheCGBros Presents ""RELATIVITY"" by Guillaume ...",20409,2022,Solid Performers,channel1
8,15,-1hANFUO3yc,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Short Films : ""TOB"" - by Andres Galeano",2014-03-18 00:31:30+00:00,"Watch this VFX short Film, by the talented And...",10370,2014,Solid Performers,channel1
9,16,-1nkzRrnVfA,UCW6NyJ6oFLPTnx7iGRZXDDg,"Horror short film about a cursed cemetery | ""T...",2021-12-15 14:30:22+00:00,Sara and Ali are looking through photo's from ...,228067,2021,Viral Hits,channel2


In [8]:
def feature_percentage_dict(df, feature_name):
    """
    Retourne un dictionnaire {classe: pourcentage} pour une colonne du DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame d'entrée
        feature_name (str): nom de la colonne à étudier
        
    Returns:
        dict: {valeur_unique: pourcentage}
    """
    counts = df[feature_name].value_counts(dropna=False)
    percentages = (counts / len(df) * 100).to_dict()
    return percentages

features_dict=feature_percentage_dict(data_processed, "view_classes")
features_dict






{'Solid Performers': 42.546182663738534,
 'Rising Stars': 23.433664901175558,
 'Viral Hits': 21.444257847823277,
 'Mega Blockbusters': 8.048055806743315,
 'Hidden Gems': 4.527838780519313}

In [11]:
from sklearn.model_selection import train_test_split

def split_function(dataframe, split_ratio=0.8, stratify_col=None, random_state=42):
    """
    Sépare le DataFrame en train et val, en respectant la distribution des classes
    si stratify_col est renseigné.
    
    Args:
        dataframe: pd.DataFrame - données à splitter
        split_ratio: float - ratio du validation set (ex: 0.2 pour 80%/20%)
        stratify_col: str ou None - le nom de la colonne à utiliser pour le split stratifié (ex: 'view_classes')
        random_state: int
    
    Returns:
        train_set, val_set: deux DataFrame disjoints
    """
    stratify = dataframe[stratify_col] if stratify_col else None
    train_set, val_set = train_test_split(
        dataframe,
        test_size=split_ratio,
        random_state=random_state,
        shuffle=True,
        stratify=stratify
    )
    return train_set, val_set

train_set, val_set = split_function(data_processed, split_ratio=0.8)
train_set.head(20)
val_set.head(20)





,Unnamed: 0,id,channel,title,date,description,views,year,view_classes,channel_real_name
10694,14106,wQv7m1i2Ljo,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D VFX Spot: ""Tougher Than You Can Imagine...",2012-10-12 06:00:10+00:00,"TheCGBros Presents ""Tougher Than You Can Imagi...",6486,2012,Rising Stars,channel1
15149,22146,oK57P_9wcww,UCTMt7iMWa7jy0fNXIktwyLA,SYLVIA | Omeleto,2021-03-20 07:16:36+00:00,A woman takes her family on a road trip.\n\n\n...,255340,2021,Viral Hits,channel29
13445,18493,3CKQlU1wSxk,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Short Film: ""Deus Ex - Human Revolutio...",2018-10-13 21:13:15+00:00,"TheCGBros Presents ""Deus Ex - Human Revolutio...",66963,2018,Solid Performers,channel1
9381,12390,pKX5GWKgsCM,UC7sDT8jZ76VLV1u__krUutA,When the extraterrestrial bass drop hits 👾👽 #s...,2023-02-24 20:56:49+00:00,"Watch ""The Signal"" by Daniel Lasker on DUST: h...",36775,2023,Solid Performers,channel9
9188,12141,oCbUdblJq74,UCrSvhrtUgpZT3EU0x1Zoy-w,CGI Showreel HD Visual Development Reel by Bra...,2015-08-15 09:28:53+00:00,CGI Showreel Brazen Animation Visual Developme...,10228,2015,Solid Performers,channel4
6128,8149,XojWbdZRrL4,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI & VFX Short Films: ""Animals Among Us: Gobl...",2018-07-11 15:31:12+00:00,"TheCGBros Presents ""Animals Among Us: Gobliffe...",9681,2018,Rising Stars,channel1
2993,3985,FvJSvk4XG_g,UCpUhcwq3oB7HKvrNeQmxJsg,Magic River - Comedy - SHORT FILM,2015-12-02 20:50:00+00:00,A wish is granted at Magic River.,930,2015,Hidden Gems,channel5
11729,15323,uZVYqv6rU_4,UCMOB6uDg7e-h8OuCw8dK2_Q,Horror Short Film “The Invaders” | ALTER Exclu...,2019-07-24 16:00:45+00:00,"On her way home, Jayla, a Muslim girl, notices...",471727,2019,Viral Hits,channel10
12534,16192,J_QBw-Cz9Gc,UCW6NyJ6oFLPTnx7iGRZXDDg,Seeing the world through the eyes of a child |...,2021-10-16 13:30:03+00:00,"Jacco is a scientist, an explorer, a musician,...",7163,2021,Rising Stars,channel2
7089,9403,cRuPO1mDGP0,UCiCnPY--pbn5S8JkJdV2PbQ,Lal Pan Bibi | লাল পান বিবি | Bengali Movie Pa...,2012-11-26 14:56:59+00:00,The Bengali Film Lal Pan Bibi : লাল পান বিবি ব...,80687,2012,Solid Performers,channel7


In [13]:
print(feature_percentage_dict(train_set, "view_classes"))
print(feature_percentage_dict(val_set, "view_classes"))


print("len train set", len(train_set))
print("len val set", len(val_set))



{'Solid Performers': 42.680662091239405, 'Rising Stars': 23.027856277755347, 'Viral Hits': 21.58255954784013, 'Mega Blockbusters': 8.187323375050465, 'Hidden Gems': 4.521598708114655}
{'Solid Performers': 42.008395221181786, 'Rising Stars': 25.056506296415886, 'Viral Hits': 20.891185017759124, 'Mega Blockbusters': 7.491120439134646, 'Hidden Gems': 4.5527930255085565}
len train set 12385
len val set 3097


## TEST BERT

In [11]:
import pandas as pd

def process_youtube_csv(csv_path):
    # Chargement du fichier CSV
    df = pd.read_csv(csv_path)
    
    # ------ 1. Ajout de la colonne 'view_classes' ------
    view_thresholds = [
        0,        # 0 vues
        1000,     # 1K vues
        10000,    # 10K vues
        100000,   # 100K vues
        1000000,  # 1M vues
        float('inf')  # Infini
    ]
    labels = [
        "Hidden Gems",
        "Rising Stars",
        "Solid Performers",
        "Viral Hits",
        "Mega Blockbusters"
    ]
    def assign_view_class(views):
        for i in range(len(view_thresholds) - 1):
            if view_thresholds[i] <= views < view_thresholds[i+1]:
                return labels[i]
        return labels[-1]
    df['view_classes'] = df['views'].apply(assign_view_class)
    
    # ------ 2. Ajout de la colonne 'channel_real_name' ------
    if 'channel' in df.columns:
        unique_channels = df['channel'].unique()
        channel_mapping = {channel: f"channel{i+1}" for i, channel in enumerate(unique_channels)}
        df['channel_real_name'] = df['channel'].map(channel_mapping)
    
    # Retourner le DataFrame modifié
    return df



In [22]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# ----- Config -----
pretrained_model = "prajjwal1/bert-tiny"  # Very small model!
num_classes = 5
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----- Dummy Dataset -----
texts = [
    "I love this movie!",     # pos
    "This was awesome",       # pos
    "I hate this film",       # neg
    "Terrible experience",    # neg
    "Brilliant performance",  # pos
    "Awful and boring"        # neg
]
labels = [1, 1, 0, 0, 1, 0]

class ToyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=32):
        self.dataframe = process_youtube_csv("../dataset/train_val.csv")
        self.encodings = tokenizer(
            list(self.dataframe["title"]), 
            truncation=True, 
            padding=True, 
            max_length=max_length, 
            return_tensors="pt"
        )
        # build mapping from class string => int
        unique_classes = sorted(self.dataframe["view_classes"].unique())
        self.class2idx = {c: i for i, c in enumerate(unique_classes)}
        self.idx2class = {i: c for c, i in self.class2idx.items()}
        self.labels = torch.tensor([self.class2idx[c] for c in self.dataframe["view_classes"]], dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]   # <-- Now it's always a 0/1/2/... tensor
        return item
# ----- Custom Model -----
class MyCustomLLM(nn.Module):
    def __init__(self, base_model_path, num_classes):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_path)
        hidden_size = self.backbone.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_classes)
    def forward(self, input_ids, attention_mask):
        output = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled = output.last_hidden_state[:, 0]  # CLS token
        return self.classifier(pooled)

# ----- Prepare -----
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
dataset = ToyDataset(texts, labels, tokenizer)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

model = MyCustomLLM(pretrained_model, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ----- Training Function -----
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# ----- Inference/Prediction -----
def predict(model, texts):
    model.eval()
    encoded = tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(encoded['input_ids'], encoded['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        preds = probs.argmax(dim=1)
    return preds.cpu().tolist(), probs.cpu().tolist()



RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# ----- Before Training -----
print("Predictions before training:")
preds, probs = predict(model, texts)
for t,p in zip(texts, preds):
    print(f"{t} -> {'positive' if p==1 else 'negative'}")

# ----- Training -----
for epoch in range(1):
    loss = train_one_epoch(model, loader, optimizer, criterion)
    print(f"Epoch {epoch+1} Loss: {loss:.4f}")

# ----- After Training -----
print("\nPredictions after training:")
preds, probs = predict(model, texts)
for t,p in zip(texts, preds):
    print(f"{t} -> {'positive' if p==1 else 'negative'}")

Predictions before training:


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
